In [1]:
import locale

In [2]:
l = locale.setlocale(locale.LC_ALL, '')
l # locale.CRNCYSTR

'English_United Kingdom.1252'

In [3]:
import requests
import json
import pandas as pd
import datetime as dt

In [4]:
from ChoreTracker import utils

In [45]:
import flask_login as fl

class AppUser(fl.UserMixin):
    ...
    def __init__(self) -> None:
        super().__init__()
        
    def get_id(self) -> str:
        # do something here
        return '1'
    
my_user = AppUser()


In [46]:
my_user.get_id()



'1'

In [47]:
fl.login_user(my_user)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [33]:
utils.get_person_details(PersonID=4)

,PersonID,PersonName,RoleName,CurrentBalance,confirmedMoney,pendingMoney
0,4,Lottie,Child,480,440,35


In [29]:
roles = utils.get_roles()
default_roleid = roles[roles['RoleName'] == 'Child']['RoleID'].values[0]
utils.create_person(
        PersonName="No name specified",
        RoleType="Child"
    )
# utils.create_person()

True

In [33]:
people = utils.get_people()
people # [people["PersonName"]=="No name specified"]["PersonID"].max()

,PersonID,PersonName,RoleID,CurrentBalance
0,1,Mike,2,280
1,2,Michelle,2,235
2,3,Jake,3,0
3,4,Lottie,3,445
4,7,No name specified,3,0
5,8,No name specified,3,0


In [42]:
from hashlib import md5
d = md5("0959".encode()).hexdigest()
d

'8af288a7f3136e5653ad28f07f4d8de3'

In [45]:
utils.update_db("delete from credentials", commit=True)

In [46]:
utils.update_db(q=f""" insert into credentials
        (RoleID, PersonID, Passcode)
        VALUES
        ('2', '{None}', '{d}')
        """,
    commit=True
)

In [ ]:
utils.update_db(q=f""" update credentials
        set Passcode = '{passcode}'
        where RoldID = '2';
        """,
    commit=True
)

In [48]:
utils.query_db("""select passcode from credentials""")['Passcode'].values[0]

'8af288a7f3136e5653ad28f07f4d8de3'

In [39]:
assert md5("2341".encode()).hexdigest() == d

AssertionError: 

In [34]:
utils.get_person_details(PersonID=8)

,PersonID,PersonName,RoleName,CurrentBalance,confirmedMoney,pendingMoney


In [22]:
utils.get_chores_table(validated=False)

WHERE i.completedBy = p.PersonId AND validated=0 AND completed=1


,PersonName,ChoreDate,name,CompletedBy,ChoreInstanceID,Completed,Validated,Banked


In [5]:
PersonId=4
utils.query_db(f"""
select 
    personid,
    personName,
    Rolename,
    sum(iif (validated=1, rate, 0)) as confirmedMoney,
    sum(iif (validated=0, rate, 0)) as pendingMoney
from 
    people as p 
join 
    choreinstances as ci
    on p.PersonId = ci.CompletedBy
join
    roles as r
on p.RoleID = r.RoleID

where 
    personid = {PersonId}
group by
    personid,
    personName,
    Rolename
""")

,PersonID,PersonName,RoleName,confirmedMoney,pendingMoney
0,4,Lottie,Child,0,270


In [6]:
utils.get_roles()

,RoleID,RoleName
0,2,Parent
1,3,Child


In [7]:
utils.get_responsibilities()

,ResponsibilityID,PersonID,ChoreID
0,1,2,6
1,3,2,2
2,4,3,3
3,5,4,4
4,6,4,5
5,7,4,6
6,8,1,1
7,9,4,1
8,10,1,25
9,11,2,11


In [8]:
utils.complete_chore_instance(choreInstanceId=16, personId=4)

True

In [9]:
utils.uncomplete_chore_instance(choreInstanceId=16, )

True

In [13]:
df = utils.get_chores_table()
df[(df['Completed'] == 1) & (df['Validated'] == 0)]

,PersonName,ChoreDate,name,CompletedBy,ChoreInstanceID,Completed,Validated,Banked
9,Jake,2023-01-26,Today test,3,11,1,0,0
10,Lottie,2023-01-26,Two day yesterday test,4,12,1,0,0
11,Lottie,2023-01-26,Two week test,4,9,1,0,0
12,Michelle,2023-01-26,One week test,2,10,1,0,0
13,Mike,2023-01-26,Two week test,4,9,1,0,0
17,Lottie,2023-02-28,Two day today test,4,17,1,0,0
18,Jake,2023-03-02,Today test,3,20,1,0,0
19,Jake,2023-03-02,Daily chore,3,25,1,0,0
20,Lottie,2023-03-02,Two day today test,4,21,1,0,0
21,Lottie,2023-03-02,Two week test,4,18,1,0,0


In [15]:
utils.get_active_chores()

,ChoreID,name,schedule,start_date,start_time,window,repeats,active,ActiveToday
0,1,Two week test,1W,2023-01-05,07:00:00,4H,1,1,True
1,2,One week test,1W,2023-01-12,07:00:00,4H,1,1,True
2,3,Today test,1W,2023-01-19,07:00:00,4H,1,1,True
4,5,Two day today test,2D,2023-01-19,07:00:00,4H,1,1,True
8,11,Test chore,3D,2023-03-02,07:00,4H,1,1,True
9,25,New test chore,2D,2023-03-02,21:37,4H,1,1,True


In [23]:
ChoreID=25
chore_date = dt.datetime.now()

utils.query_db(
                f"""
                select * from choreinstances where ChoreDate='{chore_date.date()}'
                """
            )

,ChoreInstanceID,ChoreID,ChoreDate,Completed,CompletedBy,Validated,Rate,Banked,BankedDate
0,30,5,2023-03-04,0,NaN,0,45,0,None
1,31,25,2023-03-04,0,NaN,0,45,0,None
2,32,27,2023-03-04,1,3.0,0,45,0,None


In [10]:

utils.update_db(f"""
        delete from choreinstances where date(ChoreDate)=date('{chore_date.date()}') and Completed = 0
    """)

In [22]:
utils.query_db(f"""
select * from choreinstances where date(ChoreDate)=date('{chore_date.date()}') and Completed = 0
""")

,ChoreInstanceID,ChoreID,ChoreDate,Completed,CompletedBy,Validated,Rate,Banked,BankedDate
0,30,5,2023-03-04,0,None,0,45,0,None
1,31,25,2023-03-04,0,None,0,45,0,None


In [18]:
utils.update_choreinstances()

In [21]:
utils.get_person_details(PersonID=4)

,PersonID,PersonName,RoleName,confirmedMoney,pendingMoney
0,4,Lottie,Child,270,0


In [2]:
s = requests.Session()

In [3]:
api_url = 'http://localhost:5000/api/v1/resources/{}'

In [4]:
r = s.get(api_url.format('get_chores'), params={'personId': 4}).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [25]:
# r.content.decode() # 
r['results']

'{"ChoreID":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9},"name":{"0":"Two week test","1":"One week test","2":"Today test","3":"Two day yesterday test","4":"Two day today test","5":"Two day today inactive test","6":"Test API chore","7":"Test Form chore","8":"Mr Michael E Roberts"},"schedule":{"0":"1W","1":"1W","2":"1W","3":"2D","4":"2D","5":"2D","6":"2D","7":"1D","8":"1W"},"start_date":{"0":"2023-01-05","1":"2023-01-12","2":"2023-01-19","3":"2023-01-18","4":"2023-01-19","5":"2023-01-19","6":"2023-01-28","7":"2023-01-28","8":"2023-01-28"},"start_time":{"0":"07:00:00","1":"07:00:00","2":"07:00:00","3":"07:00:00","4":"07:00:00","5":"07:00:00","6":"08:00:00","7":"(datetime.time(7, 0),)","8":"10"},"window":{"0":"4H","1":"4H","2":"4H","3":"4H","4":"4H","5":"4H","6":"6H","7":"4H","8":"12H"},"repeats":{"0":1,"1":1,"2":1,"3":1,"4":1,"5":1,"6":0,"7":1,"8":0},"active":{"0":1,"1":1,"2":1,"3":1,"4":1,"5":0,"6":1,"7":0,"8":1}}'

In [26]:
df = pd.DataFrame(json.loads(r['results']))
# list(zip(df['ChoreInstanceID'], df['name'], df['Completed']))
df

,ChoreID,name,schedule,start_date,start_time,window,repeats,active
0,1,Two week test,1W,2023-01-05,07:00:00,4H,1,1
1,2,One week test,1W,2023-01-12,07:00:00,4H,1,1
2,3,Today test,1W,2023-01-19,07:00:00,4H,1,1
3,4,Two day yesterday test,2D,2023-01-18,07:00:00,4H,1,1
4,5,Two day today test,2D,2023-01-19,07:00:00,4H,1,1
5,6,Two day today inactive test,2D,2023-01-19,07:00:00,4H,1,0
6,7,Test API chore,2D,2023-01-28,08:00:00,6H,0,1
7,8,Test Form chore,1D,2023-01-28,"(datetime.time(7, 0),)",4H,1,0
8,9,Mr Michael E Roberts,1W,2023-01-28,10,12H,0,1


In [104]:
for i in [9, 10,11,12]:
    s.post(api_url.format('uncomplete_chore_instance'), params={'choreInstanceId': i}) # .json()

In [88]:
r

<Response [200]>

In [7]:
#    name: str = None,
#    schedule: str = '1D',
#    start_date: dt.date = dt.date.today(),
#    start_time: dt.time = dt.time(7),
#    window: str = '4H',
#    repeats: bool = True,
#    active: bool = True,

s.post(api_url.format('create_chore'), 
    headers={
            "Content-type": "application/x-www-form-urlencoded; charset=UTF-8"
        },
    data={
            'name': 'Test API chore',
            'schedule': '2D',
            'start_date': dt.date.today() + dt.timedelta(days=1),
            'start_time': dt.time(8),
            'window': '6H',
            'repeats': False,
            'active': True
        }
    )

<Response [200]>